选择性能估计的方法通常取决于建模的数据。例如，当我们可以假设观察数据之间存在独立性和相同分布（i.i.d.）时，**交叉验证**通常是最合适的方法[1]。

在一般的机器学习建模过程中，我们通常采用如图（a）一样的K折交叉验证方法来评估模型性能或者调整模型的超参数。然而，时间序列数据存在时序自相关性，所以测试数据是不能出现在训练数据之前的，否则将会造成数据泄露！

因此，我们在HyperTS中内置了三种交叉验证策略来应对时间序列预测任务的模型评估。具体如下：

- 1. 如图(b)所示 *preq-bls*: 数据被分割成n个块。在初始迭代，只有前两个块被使用，第一个块作为训练集，第二个块作为验证集。在接下来的迭代中，第二个块合并进第一个块一起作为训练集，第三个块作为验证集。这个过程一直将所有的块用完为止。

- 2. 如图(c)所示 *preq-slid-bls*: 无需在每次迭代后合并块（增长窗口），以滑动窗口的方式忘记较旧的块。当过去的数据被弃用时，通常会采用这种想法，这在非静态环境中比较常见。

- 3. 如图(d)所示 *preq-bls-gap*: 在训练集块和验证集块之间引入了间隙块。这个想法背后的基本原理是增加训练集和测试集之间的独立性。

对于时间序列分类和回归任务，因为是它们是在样本层次来分割训练集和验证集的，故不涉及数据泄露。我们采用了如图(a)所示的交叉验证方式。

![cv](../images/cv.png)

参考文献

[1] Cerqueira V, Torgo L, Mozetič I. Evaluating time series forecasting models: An empirical study on performance estimation methods[J]. Machine Learning, 2020, 109(11): 1997-2028.

**使用示例:**

#### 1. 准备数据

In [1]:
from hyperts.datasets import load_network_traffic
from sklearn.model_selection import train_test_split

In [2]:
df = load_network_traffic(univariate=True)
train_data, test_data = train_test_split(df, test_size=168, shuffle=False)

#### 2. 创建实验并运行它

In [3]:
from hyperts import make_experiment

设置参数 ``cv=True``, ``num_folds`` 控制折数。

In [4]:
experiment = make_experiment(train_data=train_data.copy(),
                             task='forecast',
                             mode='dl',
                             timestamp='TimeStamp',
                             covariates=['HourSin', 'WeekCos', 'CBWD'],
                             forecast_train_data_periods=24*12,
                             max_trials=5,
                             cv=True,
                             num_folds=3)
model = experiment.run()

In [5]:
model.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('data_preprocessing',
                 TSFDataPreprocessStep(covariate_cleaner=CovariateTransformer(covariables=['HourSin',
                                                                                           'WeekCos',
                                                                                           'CBWD'],
                                                                              data_cleaner_args={'correct_object_dtype': False,
                                                                                                 'int_convert_to': 'str'}),
                                       covariate_cleaner__covariables=['HourSin',
                                                                       'WeekCos',
                                                                       'CBWD'],
                                       covariate_cleaner__data_cleaner_args={'correct_object_dtype': False,
                

#### 3. 推理与评估

In [6]:
X_test, y_test = model.split_X_y(test_data.copy())
forecast = model.predict(X_test)
results = model.evaluate(y_true=y_test, y_pred=forecast)
results

,Metirc,Score
0,mae,0.5523
1,mse,0.4975
2,rmse,0.7053
3,mape,0.3192
4,smape,0.2533
